# 0- Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    from amplpy import AMPL, tools
except ImportError:
    # Caso ocorra um erro ao importar, instala amplpy e tenta importar novamente
    %pip install amplpy
    from amplpy import AMPL, tools

In [3]:
licenca ="default"

ampl = tools.ampl_notebook(
    modules=["highs", "gokestrel", "gurobi", "cplex"],  # Módulos a serem instalados
    license_uuid=licenca,  # Licença a ser utilizada
    g=globals()  # Instancia o objeto AMPL e registra os comandos mágicos
)


# 1- Base

A base será o ordenamento dos itens para as caixas de cada *chave_loja_buffer_onda*

Por hora, testaremos com dados simulados

# 2- Modelagem

Modelagem matemática 

Dado que temos $m$ caixas para enviar a $l$ rotas, cada caixa $j$ com um volume $v$ e uma quantidade de itens $np_{j}$, para todo $j \in \{1,...,m\}$, e um número $s$ de rotas com capacidade volumétrica $cv_{l}$ distinta para cada rota $l$ , então temos:

$$
\begin{aligned}
\text{maximizar} \quad &\sum_{j = 1}^{m} \sum_{l = 1}^{s} np_{lj} y_{l j} \\
\text{sujeito a} \quad &\sum_{j = 1}^{m} v * y_{l j} \leq cv_{l} \quad \forall l \in \{1,...,s\}\\
&y_{l j} \in \{0,1\} \quad \forall j \in \{1,...,m\}, l \in \{1,...,s\}
\end{aligned}
$$

Seja $y_{lj}$​: variável binária que indica se a caixa $j$ é enviada na rota $l$ (1 se sim, 0 se não).


Queremos definir quais caixas serão selecinadas em cada rota

FIFO: Definindo o conjunto \( O_d \) das ordens de produção com a data mais antiga \( d \):

\[ O_d = \{ o \in O \mid D(o) = d \} \]

2. Definindo a sequência de subconjuntos \( O_{d+i} \), onde \( i \geq 0 \):

\[ O_{d+i} = \{ o \in O \mid D(o) = d+i \} \]

3. Formalizando a seleção das ordens de produção para empacotamento, priorizando as datas mais antigas:

\[ P(O) = \bigcup_{i=0}^{\infty} O_{d+i} \]



In [4]:
%%ampl_eval
reset;

param m; # Número de caixas
param s; # Número de rotas
param v; # Volume de cada caixa
param D; # Big number

set J = 1..m ordered; # Conjunto de caixas
set L = 1..s ordered; # Conjunto de rotas
  
param np {L,J} >= 0;  # Número de pacotes em cada caixa
param cv {L} >= 0;  # Capacidade de cada rota

var y {L, J} binary;

# Função objetivo: Maximizar o número total de caixas enviadas em cada rota
maximize num_caixas:  sum {j in J, l in L} np[l,j] * y[l,j];

# Restrição: A soma do volume das caixas enviadas em cada rota não pode exceder a capacidade da rota
s.t. R1_capacidade_Rota {l in L}:
    sum {j in J} v * y[l,j] <= cv[l];


In [5]:
# param m;  # Número de caixas
# param s;  # Número de rotas

# set J = 1..m ordered;  # Conjunto de caixas
# set L = 1..s ordered;  # Conjunto de rotas

# # Conjunto específico de caixas para cada rota
# set CJ {l in L} within J;

# param v {J} >= 0;  # Volume de cada caixa
# param d {J} >= 0;  # Demanda de cada caixa
# param cv {L} >= 0;  # Capacidade de cada rota

# var y {L, J} binary;  # Variável binária indicando se a caixa j é enviada na rota l

# # Função objetivo: Maximizar a demanda total atendida
# maximize demanda_atendida: sum {l in L, j in CJ[l]} d[j] * y[l,j];

# # Restrição 1: A soma do volume das caixas enviadas em cada rota não pode exceder a capacidade da rota
# subject to R1_capacidade_Rota {l in L}:
#     sum {j in CJ[l]} v[j] * y[l,j] <= cv[l];

# # Restrição 2: Cada caixa pode ser enviada em no máximo uma rota
# subject to R2_caixas_uma_rota {j in J}:
#     sum {l in L: j in CJ[l]} y[l,j] <= 1;

# # Dados para exemplo (opcional)
# data;
# param m := 5;  # Número de caixas
# param s := 3;  # Número de rotas

# param v :=
# 1 2
# 2 3
# 3 1
# 4 4
# 5 2;  # Volume de cada caixa

# param d :=
# 1 5
# 2 10
# 3 3
# 4 8
# 5 4;  # Demanda de cada caixa

# param cv :=
# 1 5
# 2 4
# 3 6;  # Capacidade de cada rota

# set CJ[1] := 1 2 3;
# set CJ[2] := 2 4;
# set CJ[3] := 1 3 5;

# # Resolver o problema
# solve;

# # Exibir resultados
# display y;
# display demanda_atendida;


## 2.1- Importar dados

# 2.2- Entrada de dados no modelo

In [9]:
s=50
m=8

ampl.param["m"] = s
ampl.param["s"] = m


np_ = {}
np_[1] = np.linspace(20, 100, 20, dtype=int)
np_[2] = np.linspace(50, 100, 50, dtype=int)
np_[3] = np.linspace(50, 100, 30, dtype=int)
np_[4] = np.linspace(10, 100, 35, dtype=int)
np_[5] = np.linspace(55, 100, 15, dtype=int)
np_[6] = np.linspace(60, 100, 18, dtype=int)
np_[7] = np.linspace(20, 100, 20, dtype=int)
np_[8] = np.linspace(22, 100, 50, dtype=int)

max_length = max(len(np_[i]) for i in range (1,8))
for i in range (1,3):
    np_[i] = np.pad(np_[i], (0, max_length - len(np_[i])), mode='constant')

v = 110

cv = [300]*3 + [800]*5

ampl.param["v"] = v
ampl.param['np'] = {(l + 1, j + 1): np_[l, j] for l in range(s) for j in range(m)}
ampl.param["cv"] = cv

KeyError: (0, 0)

In [21]:
aa = {}
aa[1] = np.linspace(20, 100, 50, dtype=int)
aa[2] = np.linspace(50, 100, 20, dtype=int)

# Encontrar o tamanho máximo dos arrays
max_length = max(len(aa[i]) for i in range (1,3))

# Preencher os arrays com zeros para que todos tenham o mesmo tamanho
for i in range (1,3):
    aa[i] = np.pad(aa[i], (0, max_length - len(aa[i])), mode='constant')

# Criar o DataFrame
df = pd.DataFrame(aa)
df

,1,2
0,20,50
1,21,52
2,23,55
3,24,57
4,26,60
5,28,63
6,29,65
7,31,68
8,33,71
9,34,73


In [7]:
np_ = {}
np_[1] = np.linspace(20, 100, 20, dtype=int)
np_[2] = np.linspace(50, 100, 50, dtype=int)
np_[3] = np.linspace(50, 100, 30, dtype=int)
np_[4] = np.linspace(10, 100, 35, dtype=int)
np_[5] = np.linspace(55, 100, 15, dtype=int)
np_[6] = np.linspace(60, 100, 18, dtype=int)
np_[7] = np.linspace(20, 100, 20, dtype=int)
np_[8] = np.linspace(22, 100, 50, dtype=int)

max_length = max(len(np_[i]) for i in range (1,8))
for i in range (1,3):
    np_[i] = np.pad(np_[i], (0, max_length - len(np_[i])), mode='constant')

In [10]:
aa

{1: range(20, 100, 20), 2: range(50, 100, 20)}

In [ ]:
len(v)

In [ ]:
sum(cv)

In [ ]:
sum(np)

## Solver

In [ ]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=240"
ampl.solve()

In [ ]:
%%ampl_eval
display {j in J, l in L} y[j,l];